In [1]:
from future.builtins import next
import os
import csv
import re
import collections
import logging
import optparse
from numpy import nan
import dedupe
from unidecode import unidecode
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
import json
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType,FloatType

# ## Logging

# Dedupe uses Python logging to show or suppress verbose output. Added for convenience.
# To enable verbose logging, run `python examples/csv_example.py -v`


print("PySpark Application Started")

spark = SparkSession \
    .builder \
    .appName("dedupe") \
    .master("local[*]") \
    .getOrCreate()

print(spark.sparkContext.appName)
# ## Setup


PySpark Application Started


22/09/19 15:34:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


dedupe


In [2]:
input_file = '/home/abhishekks446/pyspark/my_project_dir/dedupe_unittesting/INPUT/csv/data1.csv'
# training_file = '/home/abhishekks446/pyspark/my_project_dir/Untitled Folder/settings/csv_example_training.json'
# output_file = '/home/abhishekks446/pyspark/my_project_dir/Untitled Folder/settings/csv_example_output.csv'
# settings_file = '/home/abhishekks446/pyspark/my_project_dir/Untitled Folder/settings/csv_example_learned_settings'


primart_key="unique_id"


df= spark.read.option("header","true").csv(input_file)

In [3]:
df.count()

9986

In [3]:
df.printSchema()

root
 |-- unique_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- store_number: string (nullable = true)
 |-- store_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- store_location: string (nullable = true)
 |-- county_number: string (nullable = true)
 |-- county: string (nullable = true)
 |-- category: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- vendor_number: string (nullable = true)
 |-- vendor_name: string (nullable = true)
 |-- item_number: string (nullable = true)
 |-- item_description: string (nullable = true)
 |-- pack: string (nullable = true)
 |-- bottle_volume_ml: string (nullable = true)
 |-- state_bottle_cost: string (nullable = true)
 |-- state_bottle_retail: string (nullable = true)
 |-- bottles_sold: string (nullable = true)
 |-- sale_dollars: string (nullable = true)
 |-- volume_sold_liters: string (nullable = true)
 |

In [4]:
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    import unidecode
    # column = column.decode("utf8")
    column = unidecode.unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column :
        column = None
    return column
def readData():
    """
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID and each value is dict
    """
    
    data_d = {}
    dfn=df.na.fill("")
    results = dfn.toJSON().map(lambda j: json.loads(j)).collect()
    for row in results:
        clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
        row_id = row[primart_key]
        data_d[row_id] = dict(clean_row)

    return data_d

data_d = readData()


In [5]:
# fields = [
# {'field' : 'product', 'type': 'String'},
# {'field' : 'category', 'type': 'Exact', 'has missing' : True},
# {'field' : 'brand', 'type': 'Exact', 'has missing' : True}
# ]

fields =[
         {
           "field": "store_name",
           "type": "String"
         },
         {
           "field": "zip_code",
           "type": "String"
         },
         {
           "field": "address",
           "type": "String"
         }
       ]
deduper = dedupe.Dedupe(fields)


In [6]:
deduper.prepare_training(data_d)

In [7]:
dedupe.console_label(deduper)


store_name : hy-vee fast & fresh express- bettendorf
zip_code : 57222
address : 3470 middle road

store_name : hy-vee fast & fresh express- bettendorf
zip_code : 57222
address : 3470 middle road

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


y


store_name : hy-vee #3 food & drugstore / davenport
zip_code : 52807
address : 1823 e kimberly rd

store_name : hy-vee #4 / davenport
zip_code : 52807
address : 4064 e 53rd st

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


store_name : hy-vee #4 / davenport
zip_code : 52807
address : 4064 e 53rd st

store_name : hy-vee #3 food & drugstore / davenport
zip_code : 52807
address : 1823 e kimberly rd

1/10 positive, 1/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

store_name : hy-vee #4 / davenport
zip_code : 52807
address : 4064 e 53rd st

store_name : hy-vee #3 food & drugstore / davenport
zip_code : 52807
address : 1823 e kimberly rd

11/10 positive, 11/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


store_name : hy-vee #3 food & drugstore / davenport
zip_code : 52807
address : 1823 e kimberly rd

store_name : hy-vee #3 food & drugstore / davenport
zip_code : 52807
address : 1823 e kimberly rd

11/10 positive, 12/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling


In [ ]:
# pair = deduper.uncertain_pairs()
# print(pair)


In [9]:
# dedupe.console_label(deduper)

In [8]:
deduper.train()

In [9]:
training_file = '/home/abhishekks446/pyspark/my_project_dir/dedupe_unittesting/INPUT/csv/label.json'


In [10]:
with open(training_file, 'w') as tf:
            deduper.write_training(tf)

In [13]:
clustered_dupes = deduper.partition(data_d, 0.5)


In [15]:
cluster_membership = {}
cluster_id = 0
for (cluster_id, cluster) in enumerate(clustered_dupes):
    id_set, scores = cluster
    cluster_d = [data_d[c] for c in id_set]
    canonical_rep = dedupe.canonicalize(cluster_d)
    for record_id, score in zip(id_set, scores) :
        cluster_membership[record_id] = {
            "cluster id" : cluster_id,
            "canonical representation" : canonical_rep,
            "confidence": score
        }

singleton_id = cluster_id + 1

reader= df
ls=[]
for row in reader.collect():
    
    row_id = (row[primart_key])
    if row_id in cluster_membership :
        cluster_id = cluster_membership[row_id]["cluster id"]
        confidence_s=cluster_membership[row_id]['confidence']
        g = str(float("{0:.2f}".format(confidence_s)))
        d_list=[row_id,cluster_id,g]

        ls.append(d_list)

columns = ["row_id", "cluster_id","confidence_score"]    
dataframe = spark.createDataFrame(ls, columns,FloatType())
new_df=reader.join(dataframe ,reader.unique_id ==  dataframe.row_id,"inner")
new_df.coalesce(1).write.format("csv").option("header","true").mode("overwrite").save("/home/abhishekks446/data/big_basket_10k/output/")